In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from supabase import create_client, Client
from tqdm import tqdm
import json
import time
import logging
import os
from dotenv import load_dotenv
from rich.console import Console
from rich.progress import Progress, BarColumn, TextColumn, TimeRemainingColumn
from datetime import datetime
import re
import subprocess

In [2]:
# Load variables from .env into the environment
load_dotenv()

# Read variables
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

In [3]:
# Initialize client
supabase: Client = create_client(supabase_url, supabase_key)

In [4]:
table_name = "autoscout_car_adverts"

In [5]:
response = supabase.table(table_name).select("*").limit(1).execute()

In [6]:
response

APIResponse[~_ReturnT](data=[{'id': 950160, 'car_id': '52f0d4a7-49ad-41e9-8ac2-99b60f5d8b25', 'scraped_at': '2025-09-22T21:57:44.858731', 'make': 'nissan', 'model': 'micra', 'fuel_type': 'b', 'first_registration': '06-2013', 'mileage': 94301, 'post_code_raw': '5711 DJ', 'listing_price': 6200, 'post_code': '5711DJ', 'transmission': None, 'fuel_text': 'Benzine', 'power_text': '59 kW (80 PK)', 'power_kw': 59, 'power_pk': 80, 'model_type': '1.2 30th. Anniversary', 'model_text': 'Nissan Micra', 'range_raw': None, 'range_general': None, 'range_urban': None}], count=None)

In [7]:
response.data

[{'id': 950160,
  'car_id': '52f0d4a7-49ad-41e9-8ac2-99b60f5d8b25',
  'scraped_at': '2025-09-22T21:57:44.858731',
  'make': 'nissan',
  'model': 'micra',
  'fuel_type': 'b',
  'first_registration': '06-2013',
  'mileage': 94301,
  'post_code_raw': '5711 DJ',
  'listing_price': 6200,
  'post_code': '5711DJ',
  'transmission': None,
  'fuel_text': 'Benzine',
  'power_text': '59 kW (80 PK)',
  'power_kw': 59,
  'power_pk': 80,
  'model_type': '1.2 30th. Anniversary',
  'model_text': 'Nissan Micra',
  'range_raw': None,
  'range_general': None,
  'range_urban': None}]

## Scraping

In [8]:
base_url = "https://www.autoscout24.nl/lst"

In [9]:
params = {
    "atype": "C",
    "cy": "NL",
    "damaged_listing": "exclude",
    "desc": "1",
    "powertype": "kw",
    "sort": "age",
    "source": "homepage_search-mask",
    "ustate": "N,U",
    "kmfrom":0,
    "kmto":1000,
    "pricefrom":0,
    "priceto":10000,
    "page": 1  # start page
}

In [10]:
price_vec = np.array(
        [0, 500, 650, 1e9])
km_vec = np.array(
        [0, 1, 2, 1e9])

In [11]:
# price_vec = np.array(
#         [0, 500, 650, 700, 750, 850, 1000, 1100, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250, 3500, 4000, 4500,
#          5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000,
#          13500, 14000, 14500, 15000, 15500, 16000, 16500, 17000, 17500, 18000, 18500, 19000, 19500, 20000, 20500, 21000,
#          21500, 22000, 22500, 23000, 24000, 24500, 25000, 26000, 27000, 28000, 28500, 29000, 30000, 31000, 32000, 33000,
#          34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000,
#          50000, 52000, 54000, 56000, 58000, 60000, 62000, 64000, 66000, 68000, 70000, 75000, 80000, 85000, 90000, 95000,
#          100000, 150000, 1e9])
# km_vec = np.array(
#         [0, 1, 2, 5, 10, 15, 20, 50, 100, 200, 500, 1000, 2000, 3000, 5000, 10000, 15000, 20000, 25000, 30000, 35000,
#          40000, 45000, 50000, 55000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 145000, 150000,
#          155000, 160000, 170000, 180000, 190000, 200000, 210000, 220000, 230000, 240000, 260000, 280000, 300000, 350000,
#          400000, 1e9])

In [12]:
price_vec

array([0.0e+00, 5.0e+02, 6.5e+02, 1.0e+09])

In [13]:
km_vec

array([0.e+00, 1.e+00, 2.e+00, 1.e+09])

In [14]:
def is_valid_format(s, pattern):
    return bool(re.fullmatch(pattern, s))

def extract_car_data(car_html):
    # Extract correct mileage
    try:
        data_mileage = float(car_html.get("data-mileage"))
    except (ValueError, TypeError):
        data_mileage = -1

    # Extract correct listing price
    try:
        listing_price = float(car_html.get("data-price"))
    except (ValueError, TypeError):
        listing_price = -1

    # Extract postcode
    raw_postcode = car_html.get("data-listing-zip-code")
    try:
        postcode = raw_postcode[0:4] + raw_postcode[-2:].upper()
        if not is_valid_format(postcode, pattern):
            postcode = None
    except:
        postcode = None

    # Extract the desired details by their data-testid attributes
    transmission = car_html.find("span", {"data-testid": "VehicleDetails-transmission"})
    fuel = car_html.find("span", {"data-testid": "VehicleDetails-gas_pump"})
    power = car_html.find("span", {"data-testid": "VehicleDetails-speedometer"})

    # Get the text values, stripping whitespace
    transmission_text = transmission.get_text(strip=True) if transmission else None
    fuel_text = fuel.get_text(strip=True) if fuel else None
    power_text = power.get_text(strip=True) if power else None

    kw_value = None
    pk_value = None

    if power_text:
        # Extract numbers: first one before 'kW', second inside parentheses
        match = re.search(r"(\d+)\s*kW.*\((\d+)\s*PK\)", power_text)
        if match:
            kw_value = float(match.group(1))
            pk_value = float(match.group(2))

    # Find car and model specifics
    title_element = car_html.find("span", class_="ListItem_title_bold__iQJRq")
    model_text = title_element.get_text(strip=True) if title_element else None
    version_element = car_html.find("span", class_="ListItem_version__5EWfi")
    version_text = version_element.get_text(strip=True) if version_element else None

    # Find the actieradius span by aria-label
    actieradius_element = car_html.find("span", attrs={"aria-label": "actieradius"})
    actieradius_text = actieradius_element.get_text(strip=True) if actieradius_element else None

    # Extract both numeric values as floats
    ranges = [float(num) for num in
              re.findall(r"\d+(?:\.\d+)?", actieradius_text)] if actieradius_text else []

    general_range = ranges[0] if len(ranges) > 0 else None
    urban_range = ranges[1] if len(ranges) > 1 else None

    car_info = {
        "car_id": car_id,
        "make": car_html.get("data-make"),
        "model": car_html.get("data-model"),
        "first_registration": car_html.get("data-first-registration"),
        "fuel_type": car_html.get("data-fuel-type"),
        "mileage": data_mileage,
        "post_code_raw": raw_postcode,
        "post_code": postcode,
        "listing_price": listing_price,
        "transmission": transmission_text,
        "fuel_text": fuel_text,
        "power_text": power_text,
        "power_kw": kw_value,
        "power_pk": pk_value,
        "model_text": model_text,
        "model_type": version_text,
        "range_raw": actieradius_text,
        "range_general": general_range,
        "range_urban": urban_range
    }
    
    return car_info

In [15]:
count_added = 0
cars_to_insert = []
car_ids_in_upsert = set()
car_ids_in_all_upserts = set()
batch_size = 500
page_limit_autoscout = 20
refresh_rate_cars_in_database = 500

print("Fetching existing car IDs from the database...")
response = supabase.table(table_name).select("car_id").execute()
car_ids_in_database = {d['car_id'] for d in response.data}
print(f"Found {len(car_ids_in_database)} existing car IDs.")

Fetching existing car IDs from the database...
Found 279694 existing car IDs.


In [16]:
# Get current date and time
now = datetime.now()

# Format it as YYYY-MM-DD_HH-MM-SS
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# The level is set to INFO, so all messages from INFO and above will be recorded.
logging.basicConfig(filename=f"../logging/script_log_{timestamp}.log", level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logging.info('Script started.')

# Initialize a rich Console object
console = Console()

# IMPORTANT: if is_processing
is_processing_all = 1

# Pattern for postcode
pattern = r'^\d{4}[A-Z]{2}$'

# --- Main Loops ---
# Custom Progress display
with Progress(
    TextColumn("[progress.description]{task.description}"),
    BarColumn(),
    TextColumn("[progress.percentage]{task.percentage:>3.0f}%"),
    TimeRemainingColumn(),
    console=console
) as progress:
    
    # Outer loop over price range
    task_price = progress.add_task(
        "[green]Processing price ranges...", total=len(price_vec[:-1]-1)
    )

    for k, price in enumerate(price_vec[:-1]):
        params['pricefrom'] = round(price_vec[k])
        params['priceto'] = round(price_vec[k+1])

        if k % refresh_rate_cars_in_database == 0:
            response = supabase.table(table_name).select("car_id").execute()
            car_ids_in_database = {d['car_id'] for d in response.data}

        # Inner loop over mileage
        task_mileage = progress.add_task(
            f"[cyan]  Processing mileage {round(km_vec[0])}-{round(km_vec[-1])}...",
            total=len(km_vec[:-1]-1)
        )

        for j, km in enumerate(km_vec[:-1]):
            params['kmfrom'] = round(km_vec[j])
            params['kmto'] = round(km_vec[j+1])

            # Flag to check if the page loop completes fully
            page_limit_reached = True

            # Innermost loop over pages
            for i in range(page_limit_autoscout):
                params['page'] = i + 1
                html = requests.get(base_url, params=params).text
                soup = BeautifulSoup(html, "html.parser")
                car_listings = soup.find_all("article", class_="cldt-summary-full-item")

                if not car_listings:
                    # If no listings are found, the loop breaks early.
                    page_limit_reached = False
                    break

                for car in car_listings:
                    
                    # Check if car is already in database
                    car_id = car.get("id")
                    if car_id not in car_ids_in_all_upserts and ((car_id not in car_ids_in_database) or is_processing_all):

                        car_info = extract_car_data(car)
                        
                        cars_to_insert.append(car_info)
                        car_ids_in_database.add(car_id)
                        car_ids_in_upsert.add(car_id)
                        car_ids_in_all_upserts.add(car_id)

                        # Update database in batches
                        if len(cars_to_insert) >= batch_size:
                            console.log(f"Inserting {len(cars_to_insert)} cars to the database...")
                            logging.info(f"Inserting {len(cars_to_insert)} cars to the database...")
                            supabase.table(table_name).upsert(cars_to_insert, ignore_duplicates=True).execute()
                            count_added += len(cars_to_insert)
                            cars_to_insert = []
                            car_ids_in_upsert = set()

                time.sleep(0.01)

            # Check and log if the page limit was reached for this mileage-price combination
            if page_limit_reached:
                console.log(f"Reached page limit for price: {params['pricefrom']}-{params['priceto']} and mileage: {params['kmfrom']}-{params['kmto']}")
                logging.info(f"Reached page limit for price: {params['pricefrom']}-{params['priceto']} and mileage: {params['kmfrom']}-{params['kmto']}")
                
            # Update the mileage task for each mileage range
            progress.update(task_mileage, advance=1)
        
        # Mark the mileage task as complete and remove it
        progress.remove_task(task_mileage)

        # Update the price task for each price range
        progress.update(task_price, advance=1)

    # Mark the price task as complete and remove it
    # progress.remove_task(task_price)

# --- Final Batch Insert ---
# Insert any remaining cars after all loops have finished
if cars_to_insert:
    console.log(f"Inserting final {len(cars_to_insert)} cars to the database...")
    supabase.table(table_name).upsert(cars_to_insert, ignore_duplicates=True).execute()
    count_added += len(cars_to_insert)

# console.log(f"\nTotal cars added to the database: {count_added}")
logging.info(f"\nTotal cars added to the database: {count_added}")
logging.info('Script finished successfully.')

Output()

[21:06:42] Reached page limit for price: 0-500 and mileage: 2-1000000000                           ]8;id=314173;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py\1575508218.py]8;;\:]8;id=746821;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py#95\95]8;;\

[21:07:11] Inserting 500 cars to the database...                                                   ]8;id=791109;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py\1575508218.py]8;;\:]8;id=622610;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py#84\84]8;;\

[21:07:12] Reached page limit for price: 650-1000000000 and mileage: 0-1                           ]8;id=775538;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py\1575508218.py]8;;\:]8;id=263067;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py#95\95]8;;\

[21:07:35] Reached page limit for price: 650-1000000000 and mileage: 1-2                           ]8;id=656835;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py\1575508218.py]8;;\:]8;id=647569;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py#95\95]8;;\

[21:07:57] Reached page limit for price: 650-1000000000 and mileage: 2-1000000000                  ]8;id=686118;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py\1575508218.py]8;;\:]8;id=256243;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py#95\95]8;;\

           Inserting final 462 cars to the database...                                            ]8;id=359562;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py\1575508218.py]8;;\:]8;id=645366;file://C:\Users\david\AppData\Local\Temp\ipykernel_7732\1575508218.py#113\113]8;;\

In [17]:
count_added

1462

In [18]:
last_cars_df = pd.DataFrame(cars_to_insert)

In [19]:
last_cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   car_id              462 non-null    object 
 1   make                462 non-null    object 
 2   model               462 non-null    object 
 3   first_registration  462 non-null    object 
 4   fuel_type           462 non-null    object 
 5   mileage             462 non-null    float64
 6   post_code_raw       462 non-null    object 
 7   post_code           436 non-null    object 
 8   listing_price       462 non-null    float64
 9   transmission        0 non-null      object 
 10  fuel_text           462 non-null    object 
 11  power_text          462 non-null    object 
 12  power_kw            440 non-null    float64
 13  power_pk            440 non-null    float64
 14  model_text          462 non-null    object 
 15  model_type          462 non-null    object 
 16  range_ra

In [20]:
response = supabase.table(table_name).select("car_id", "transmission").execute()
car_ids_in_database = response.data

In [21]:
df = pd.DataFrame([d['car_id'] for d in car_ids_in_database])

In [22]:
len(df[0].unique()) - len(df)

0

In [23]:
df_all = pd.DataFrame(car_ids_in_database)

## Remove duplicates in database

In [8]:
response = supabase.table(table_name).select("id, car_id").execute()
car_ids_in_database = response.data

In [9]:
df_full = pd.DataFrame(car_ids_in_database)

In [10]:
df_full['car_id'].unique()

array(['52f0d4a7-49ad-41e9-8ac2-99b60f5d8b25',
       '48a53c89-9045-49a5-8b45-3223edbf0ab6',
       '1b9b04e0-5f60-4f1a-b0d9-d38a427b9112', ...,
       '730dfe58-38af-4303-8aa5-8d41bb8617f4',
       '8345d124-cc8e-4cec-84a0-e0345bf58b4d',
       'f6ed9002-5e9d-49a1-9a29-fae48a6fc60c'],
      shape=(346605,), dtype=object)

In [11]:
def fetch_all_rows_in_batches(
    supabase,
    table_name: str,
    columns: str = "*",
    batch_size: int = 5000,
    max_batches: int | None = None
):
    """
    Fetch all rows from a Supabase table in batches to avoid timeouts.

    Args:
        supabase: Supabase client instance
        table_name: Name of the table to query
        columns: Comma-separated column names or "*" for all
        batch_size: Number of rows per batch
        max_batches: Optional limit (for testing or large tables)

    Returns:
        List of dicts containing all rows fetched.
    """
    all_rows = []
    offset = 0
    batch_count = 0

    while True:
        try:
            response = (
                supabase.table(table_name)
                .select(columns)
                .order("car_id", desc=False)
                .range(offset, offset + batch_size - 1)
                .execute()
            )

            data = response.data

            # Stop when no more rows
            if not data:
                logging.info(f"No more rows after offset {offset}.")
                break

            all_rows.extend(data)
            offset += batch_size
            batch_count += 1

            logging.info(f"Fetched {len(data)} rows (total {len(all_rows)}).")

            # Optional: stop early if max_batches is set
            if max_batches and batch_count >= max_batches:
                logging.info(f"Reached max_batches ({max_batches}), stopping early.")
                break

        except Exception as e:
            logging.error(f"Error fetching batch starting at {offset}: {e}")
            time.sleep(2)
            break

    return all_rows

In [12]:
response = fetch_all_rows_in_batches(supabase, table_name, "id, car_id, make, listing_price", batch_size=50000)

In [17]:
response

[{'id': 1322377,
  'car_id': '00000b3b-7d33-4020-a1f4-99858043e523',
  'make': 'lynk & co',
  'listing_price': 26445},
 {'id': 129016,
  'car_id': '00001d30-9ff0-48eb-a8f2-87f3ab617417',
  'make': 'chevrolet',
  'listing_price': 1925},
 {'id': 17693,
  'car_id': '00004c30-e8cd-441b-a9ae-3f52aecedc24',
  'make': 'bmw',
  'listing_price': 72122},
 {'id': 22392,
  'car_id': '00004e9d-3234-49d0-8e0e-5802182becc3',
  'make': 'mercedes-benz',
  'listing_price': 1550000},
 {'id': 190970,
  'car_id': '000068b6-50d4-4aba-ad27-d9fe14a4c056',
  'make': 'audi',
  'listing_price': 13250},
 {'id': 78445,
  'car_id': '000097b9-0c3e-4dd3-8d42-2b8166e9f988',
  'make': 'volvo',
  'listing_price': 29950},
 {'id': 1048021,
  'car_id': '0000c675-b7ff-465f-ad5a-77c8a817fedd',
  'make': 'peugeot',
  'listing_price': 19700},
 {'id': 388799,
  'car_id': '0000cbad-c6f9-46a1-bb93-b61e42c356d3',
  'make': 'mazda',
  'listing_price': 21495},
 {'id': 1315823,
  'car_id': '0000d0c8-b216-4015-91e2-f8c90f33b9bb',
  'm

In [13]:
df_full = pd.DataFrame(response)

In [14]:
df_full

,id,car_id,make,listing_price
0,1322377,00000b3b-7d33-4020-a1f4-99858043e523,lynk & co,26445
1,129016,00001d30-9ff0-48eb-a8f2-87f3ab617417,chevrolet,1925
2,17693,00004c30-e8cd-441b-a9ae-3f52aecedc24,bmw,72122
3,22392,00004e9d-3234-49d0-8e0e-5802182becc3,mercedes-benz,1550000
4,190970,000068b6-50d4-4aba-ad27-d9fe14a4c056,audi,13250
...,...,...,...,...
346600,1331640,ffff821d-980d-4a63-88b7-609d3d2000e0,kia,31990
346601,161994,ffff8630-65ed-4fe0-8b60-1e36717d3014,tesla,34500
346602,176076,ffffa53f-5548-47b6-a4d0-df1d08b0bfdc,renault,2965
346603,14650,ffffaa06-90e3-49e8-8e25-07586b76de41,volkswagen,53568


In [15]:
df_full['car_id'].unique()

array(['00000b3b-7d33-4020-a1f4-99858043e523',
       '00001d30-9ff0-48eb-a8f2-87f3ab617417',
       '00004c30-e8cd-441b-a9ae-3f52aecedc24', ...,
       'ffffa53f-5548-47b6-a4d0-df1d08b0bfdc',
       'ffffaa06-90e3-49e8-8e25-07586b76de41',
       'ffffb78c-0efc-43d4-b480-c392615d924f'],
      shape=(346605,), dtype=object)

In [23]:
id_to_remove = df_full.loc[(df_full.duplicated(subset=['car_id'],keep="first")), 'car_id'].values

In [24]:
id_to_remove

array([], dtype=object)

In [31]:
response = (
        supabase.table(table_name)
        .delete()
        .in_("car_id", ["4cef663e-6aae-4b34-8137-1e2a11ab0b3d"])
        .execute()
    )

In [32]:
response

APIResponse[~_ReturnT](data=[{'id': 370110, 'car_id': '4cef663e-6aae-4b34-8137-1e2a11ab0b3d', 'scraped_at': '2025-08-14T12:48:22.797734', 'make': 'skoda', 'model': 'superb', 'fuel_type': '2', 'first_registration': '01-2025', 'mileage': 10, 'post_code_raw': '3846 AV', 'listing_price': 49250, 'post_code': '3846AV', 'transmission': None, 'fuel_text': None, 'power_text': None, 'power_kw': None, 'power_pk': None, 'model_type': None, 'model_text': None, 'range_raw': None, 'range_general': None, 'range_urban': None}], count=None)

In [29]:
chunk_size = 1000

for i in tqdm(range(0, len(id_to_remove), chunk_size)):
    chunk = id_to_remove[i:min(i + chunk_size, len(id_to_remove))]
    response = (
        supabase.table(table_name)
        .delete()
        .in_("id", chunk)
        .execute()
    )


0it [00:00, ?it/s]


## Check latitude and longitude

In [84]:
BASE_URL = "https://openpostcode.nl/api/address"

params = {
    "postcode": "1011AB",
    "huisnummer": '99'
}

response = requests.get(BASE_URL, params=params)

In [85]:
response.json()

{'postcode': '1011AB',
 'huisnummer': '99',
 'straat': 'De Ruijterkade',
 'buurt': 'Oosterdokseiland',
 'wijk': 'Nieuwmarkt/Lastage',
 'woonplaats': 'Amsterdam',
 'gemeente': 'Amsterdam',
 'provincie': 'Noord-Holland',
 'latitude': 52.378666,
 'longitude': 4.9054437}

## Check post code convention

In [187]:
response = supabase.table(table_name).select("id, car_id, post_code, post_code_raw").execute()

In [188]:
df_full = pd.DataFrame(response.data)

In [189]:
df_full['post_code'].sort_values().unique()

array(['0000AA', '0000AB', '0000OO', ..., '9997PG', '9998NZ', None],
      shape=(31130,), dtype=object)

In [190]:
df_full

,id,car_id,post_code,post_code_raw
0,77546,68936f70-81e9-4387-ad5c-ac6eed1f5438,8281PC,8281 PC
1,151,0d92d6f1-39e5-4606-b239-1eafa1eb65f2,2987VD,2987 VD
2,152,e148c182-bfdc-4820-befa-b7f8bb3569fe,7903BM,7903 BM
3,153,ac97cde6-8a38-4dca-a30a-bfb2286cb9ea,7513EL,7513 EL
4,154,99c8af8d-6633-46c2-81e3-012d73c34c8b,5505JA,5505 JA
...,...,...,...,...
268541,276040,acf0832f-293c-4ac5-a276-fbf410baa346,5161CB,5161 CB
268542,276041,761a9158-01b3-4044-95ee-7e5ae70b40ad,2988AD,2988 AD
268543,276042,cec055f4-e88a-4965-9238-a226e6313057,7943KC,7943KC
268544,276043,8bc1bca9-2ba0-4294-b80b-3a919effa566,9482RD,9482rd


In [166]:
batch_size = 100  # choose a batch size to avoid huge payloads
updates = []

for index, row in tqdm(df_full.iterrows(), total=df_full.shape[0]):
    if row['post_code']:  # skip if already set
        continue
    
    raw_postcode = row['post_code_raw']
    
    # Compute formatted postcode
    try:
        postcode = raw_postcode[0:4] + raw_postcode[-2:].upper()
        if not is_valid_format(postcode, pattern):
            postcode = None
    except Exception as e:
        postcode = None
    
    # Append to batch list
    updates.append({"id": row['id'], "post_code": postcode})
    
    # When batch is full, send to Supabase
    if len(updates) >= batch_size:
        supabase.table(table_name).upsert(updates).execute()
        updates = []  # clear batch

# Send remaining updates if any
if updates:
    supabase.table(table_name).upsert(updates).execute()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268546/268546 [05:58<00:00, 749.33it/s]


In [182]:
BASE_URL = "https://openpostcode.nl/api/address"

params = {
    "postcode": "4167BM",
    "huisnummer": '1'
}

response = requests.get(BASE_URL, params=params)

In [183]:
response.text

'{"error":"Postcode not found"}\n'